<a href="https://colab.research.google.com/github/carlos-alves-one/-SSDM-Coursework-2/blob/main/Regression_CW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Goldsmiths University of London
### MSc. Data Science and Artificial Intelligence
### Module: Statistics and Statistical Data Mining
### Author: Carlos Manuel De Oliveira Alves
### Student: cdeol003
### Coursework No.2

### Load the data

> Run the code in Python 3

In [1]:
# Imports the 'drive' module from 'google.colab' and mounts the Google Drive to
# the '/content/drive' directory in the Colab environment.
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


> Run the code in R

In [10]:
# Load necessary libraries for this project
library(tidyverse)    # for data manipulation and visualization

install.packages("caret")
library(caret)        # for machine learning

library(rpart)        # for decision trees

install.packages("randomForest")
library(randomForest) # for random forest algorithm

install.packages("gbm")
library(gbm)          # for gradient boosting machines

library(e1071)        # for SVM and other algorithms

# Load the 'readr' library for data import and manipulation in R
library(readr)

# Set seed for reproducibility
set.seed(123)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [12]:
# Loading the datasets using base R read.csv function
train_data <- read.csv("/content/drive/MyDrive/credit_risk/creditdefault_train.csv", sep = ";")
test_data  <- read.csv("/content/drive/MyDrive/credit_risk/creditdefault_train.csv", sep = ";")

# Displaying the first few rows of the Credit Default training dataset
head(train_data)

# Displaying the first few rows of the Credit Default test dataset
head(test_data)


,Y.X1.X2.X3.X4.X5.X6.X7.X8.X9.X10.X11.X12.X13.X14.X15.X16.X17.X18.X19.X20.X21.X22.X23
,<chr>
1,"1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0"
2,"0,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000"
3,"0,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679"
4,"0,50000,1,1,2,37,0,0,0,0,0,0,64400,57069,57608,19394,19619,20024,2500,1815,657,1000,1000,800"
5,"0,500000,1,1,2,29,0,0,0,0,0,0,367965,412023,445007,542653,483003,473944,55000,40000,38000,20239,13750,13770"
6,"0,100000,2,2,2,23,0,-1,-1,0,0,-1,11876,380,601,221,-159,567,380,601,0,581,1687,1542"


,Y.X1.X2.X3.X4.X5.X6.X7.X8.X9.X10.X11.X12.X13.X14.X15.X16.X17.X18.X19.X20.X21.X22.X23
,<chr>
1,"1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0"
2,"0,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000"
3,"0,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679"
4,"0,50000,1,1,2,37,0,0,0,0,0,0,64400,57069,57608,19394,19619,20024,2500,1815,657,1000,1000,800"
5,"0,500000,1,1,2,29,0,0,0,0,0,0,367965,412023,445007,542653,483003,473944,55000,40000,38000,20239,13750,13770"
6,"0,100000,2,2,2,23,0,-1,-1,0,0,-1,11876,380,601,221,-159,567,380,601,0,581,1687,1542"


# Problem Formulation

This project aims to predict whether a credit card holder will default on their payment in the next month. We will use a binary classification approach with the response variable Y indicating default payment (Yes = 1, No = 0). The challenge is to build and select a predictive model that performs well on unseen data, balancing the trade-off between underfitting and overfitting.

# Data Exploratory Analysis

## Initial Exploration of the Dataset

In [13]:
# Exploratory Data Analysis
# Summary of the training data
summary(train_data)

 Y.X1.X2.X3.X4.X5.X6.X7.X8.X9.X10.X11.X12.X13.X14.X15.X16.X17.X18.X19.X20.X21.X22.X23
 Length:15000                                                                        
 Class :character                                                                    
 Mode  :character                                                                    

**Given the detailed information about the variables in the dataset, we can draw some conclusions and suggest potential steps for further analysis:**

- The dataset contains 24 columns and 15,000 rows, all being integer type

## Variable Types and Conversion


- X1 (Credit Amount): This should be a numeric variable as it represents amounts in dollars.

- X2 (Gender): Since it is coded as 1 (male) and 2 (female), it is a categorical variable. Consider converting it to a factor with two levels.

- X3 (Education), X4 (Marital Status): These are categorical variables and should be converted to factors with their respective levels.

- X5 (Age): This is a numeric variable.

- X6-X11 (Repayment Status): These are ordinal categorical variables. They have an inherent order (-1, 1, 2, ..., 9) and should be treated as factors with ordered levels.

- X12-X17 (Bill Statement Amount), X18-X23 (Previous Payment Amount): These should be numeric variables representing monetary values.


## Data Integrity and Cleaning

- We need to ensure that the data types in the dataset match these specifications. They must be converted to the appropriate types if they are all currently 'character' types.

- Check for missing or unusual values. For example, ensure that only the specified categories are present in categorical variables.

### Check for Missing Values

In [14]:
# Check for missing values in the training dataset
sum(is.na(train_data))

[1] 0

The output is zero, meaning there are no missing values (NA) in the train_data dataset. Every element in the dataset has a defined, non-missing value. This is generally a good sign for data quality, as missing data can complicate analysis and require additional steps for handling.

## Data Type Conversion

Convert the variables to their appropriate data types based on the descriptions provided.

In [25]:
# Ensure dplyr is installed and loaded
if (!require(dplyr)) install.packages("dplyr")
library(dplyr)

# Load your data into train_data
# train_data <- read.csv("path_to_your_csv_file.csv")

# Check if 'X2' column exists in train_data
if("X2" %in% colnames(train_data)) {
  # Directly convert columns using dplyr's mutate and mutate_at functions
  train_data <- train_data %>%
    mutate(
      X2 = factor(X2, levels = c(1, 2), labels = c("Male", "Female")),
      X3 = factor(X3, levels = c(1, 2, 3, 4), labels = c("Graduate School", "University", "High School", "Others")),
      X4 = factor(X4, levels = c(1, 2, 3), labels = c("Married", "Single", "Others"))
    ) %>%
    mutate_at(vars(X6:X11), funs(factor(., ordered = TRUE)))
} else {
  stop("Column 'X2' not found in train_data")
}

# Rest of your code...


ERROR: ignored